In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
import cv2


/Users/mikaelvo/PyEnvs/coreml_convert/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)#, input_shape=(299, 299, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
out = Dense(2, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=out)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy')


In [ ]:
model.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
def normalize(image):
#    image2 = image / 255.
#    image2 = image2 - 0.5
#    image2 = image2 * 2.
    return image

train_datagen = ImageDataGenerator(
        rescale=1/255.,
#        preprocessing_function=normalize,
        
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        'flowers/train',
        target_size=(299, 299),
        batch_size=32,
        class_mode='categorical')

#test_datagen = ImageDataGenerator(rescale=1./255)
#validation_generator = test_datagen.flow_from_directory(
#        'data/validation',
#        target_size=(150, 150),
#        batch_size=32,
#        class_mode='binary')

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=32, epochs=2)

In [19]:
image = cv2.imread('flowers/train/Rose/test - 4.jpg')
image = cv2.resize(image, (299, 299))


In [26]:
import numpy as np
preprocessed_image = normalize(image)
model.predict(np.expand_dims(preprocessed_image, axis=0))

array([[0.49186438, 0.5081356 ]], dtype=float32)